In [1]:
import os
import numpy as np
import pandas as pd
from shutil import copyfile

from keras import backend as K
from keras import layers as L
from keras.models import Model,load_model
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping
from keras.layers import Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator

# import matplotlib.pyplot as plt

os.mkdir('./images/')
alpha = 'a'
for i in range(0, 26): 
    os.mkdir('./images/' + alpha)
    alpha = chr(ord(alpha) + 1)

rootdir = 'dataset/Braille Dataset/Braille Dataset/'
for file in os.listdir(rootdir):
    letter = file[0]
    copyfile(rootdir+file, './images/' + letter + '/' + file)

datagen = ImageDataGenerator(rotation_range=20,
                             shear_range=10,
                             validation_split=0.2)

train_generator = datagen.flow_from_directory('./images/',
                                              target_size=(28,28),
                                              subset='training')

val_generator = datagen.flow_from_directory('./images/',
                                            target_size=(28,28),
                                            subset='validation')

# from tensorflow.keras.preprocessing import image
# img=image.load_img('dataset/Braille Dataset/Braille Dataset/a1.JPG0dim.jpg')

# x=image.img_to_array(img)
# x.shape
# x=np.expand_dims(x,axis=0)
# model.predict(x)

# K.clear_session()

model_ckpt = ModelCheckpoint('BrailleNet.h5', save_best_only=True)
reduce_lr = ReduceLROnPlateau(patience=8, verbose=0)
early_stop = EarlyStopping(patience=15, verbose=1)

entry = L.Input(shape=(28, 28, 3))
x = L.SeparableConv2D(64, (3, 3), activation='relu')(entry)
x = L.BatchNormalization()(x)
x = L.MaxPooling2D((2, 2))(x)

x = L.SeparableConv2D(128, (3, 3), activation='relu')(x)
x = L.BatchNormalization()(x)
x = L.MaxPooling2D((2, 2))(x)

x = L.SeparableConv2D(256, (2, 2), activation='relu')(x)
x = L.BatchNormalization()(x)
x = L.GlobalMaxPooling2D()(x)

x = L.Dense(256)(x)
x = L.LeakyReLU()(x)
x = L.Dropout(0.5)(x)

x = L.Dense(64, kernel_regularizer=l2(2e-4))(x)
x = L.LeakyReLU()(x)
x = L.Dropout(0.5)(x)

x = L.Dense(26, activation='softmax')(x)

model = Model(entry, x)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit_generator(train_generator,
                              validation_data=val_generator,
                              epochs=666,
                              callbacks=[model_ckpt, reduce_lr, early_stop],
                              verbose=1)

model.summary()

# Load the trained model
model = load_model('BrailleNet.h5')

# Evaluate the model on the validation generator
eval_results = model.evaluate_generator(val_generator)

# Extract the accuracy from the evaluation results
accuracy = eval_results[1]

# Print the accuracy
print('Validation Accuracy: {:.2%}'.format(accuracy))

Found 1248 images belonging to 26 classes.
Found 312 images belonging to 26 classes.
Epoch 1/666


C:\Users\ishmu\anaconda3\envs\brl_env\lib\site-packages\ipykernel_launcher.py:83: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


39/39 [==============================] - 7s 156ms/step - loss: 4.2374 - accuracy: 0.0545 - val_loss: 3.2741 - val_accuracy: 0.0449 - lr: 0.0010
Epoch 2/666
39/39 [==============================] - 1s 36ms/step - loss: 3.3333 - accuracy: 0.0881 - val_loss: 3.2579 - val_accuracy: 0.0994 - lr: 0.0010
Epoch 3/666
39/39 [==============================] - 1s 36ms/step - loss: 3.0710 - accuracy: 0.1554 - val_loss: 3.2197 - val_accuracy: 0.2115 - lr: 0.0010
Epoch 4/666
39/39 [==============================] - 1s 36ms/step - loss: 2.8616 - accuracy: 0.1875 - val_loss: 3.1545 - val_accuracy: 0.1731 - lr: 0.0010
Epoch 5/666
39/39 [==============================] - 1s 37ms/step - loss: 2.5810 - accuracy: 0.2508 - val_loss: 3.0491 - val_accuracy: 0.2404 - lr: 0.0010
Epoch 6/666
39/39 [==============================] - 1s 37ms/step - loss: 2.4262 - accuracy: 0.2829 - val_loss: 2.9417 - val_accuracy: 0.3429 - lr: 0.0010
Epoch 7/666
39/39 [==============================] - 1s 38ms/step - loss: 2.1751 

39/39 [==============================] - 2s 42ms/step - loss: 0.3424 - accuracy: 0.9014 - val_loss: 0.2394 - val_accuracy: 0.9391 - lr: 1.0000e-04
Epoch 54/666
39/39 [==============================] - 2s 40ms/step - loss: 0.3333 - accuracy: 0.8966 - val_loss: 0.2196 - val_accuracy: 0.9295 - lr: 1.0000e-04
Epoch 55/666
39/39 [==============================] - 2s 40ms/step - loss: 0.3434 - accuracy: 0.8966 - val_loss: 0.2365 - val_accuracy: 0.9263 - lr: 1.0000e-04
Epoch 56/666
39/39 [==============================] - 2s 40ms/step - loss: 0.3119 - accuracy: 0.9183 - val_loss: 0.2416 - val_accuracy: 0.9295 - lr: 1.0000e-04
Epoch 57/666
39/39 [==============================] - 2s 41ms/step - loss: 0.3143 - accuracy: 0.9030 - val_loss: 0.2636 - val_accuracy: 0.9263 - lr: 1.0000e-04
Epoch 58/666
39/39 [==============================] - 2s 44ms/step - loss: 0.2986 - accuracy: 0.9054 - val_loss: 0.2068 - val_accuracy: 0.9487 - lr: 1.0000e-05
Epoch 59/666
39/39 [==============================] -

C:\Users\ishmu\anaconda3\envs\brl_env\lib\site-packages\ipykernel_launcher.py:91: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.


Validation Accuracy: 94.55%
